In [2]:
import requests
import json
import pandas as pd
from datetime import datetime

In [23]:
url = "https://beanyfood.mysapogo.com/admin/orders.json?page=1&limit=100"
cookie = "report_ver=3; _fw_crm_v=b03a9b5b-9cdb-4d81-ab85-c51c8076a2ee; _fbp=fb.1.1722010576000.953081039792325240; _gid=GA1.2.1309095844.1727093848; sapo-admin=CfDJ8DA1016STPdNrAaTdK2HgWezJvDLuzWt9H9UYVbHnKoC5hmGaDXYfOuJbnl2FkOUi066IPTm-I6LGyHyDdzk_rly-n0_HgINFzGdc_kNHddOB0G7mev2nocaLlk2YairKczE3A_tjlEKhuoqhhHMgc7CZy29u5wdsdo2shtXsq2muJGNiMPoIcpbaX8rr8rcFw9iIQuifnN-ajuFMvyBiZleauHvCDNrk6DqhjIbDrj1aKBgXsUD1KAzmt30RjuYmKTG9sANL9OTTF-IsDxxdkxt0XCWkJYjCAqSR2J3FqtcNTd9jynfoAe-Ng3-XUIgle3WHlGv96Ob9EFbzmHNllirgJVCZ7gFknv_L-1JiPN11Ac-qzeMFytofzt-EdxXMf26JZB2dfjJRmzUoFkgsZVEY2vb7cfN02lU7nNLbMTCQ9VXAVf2Oid9Y1as4ZDOhxcHscCPYpWPxr-IFJ_XotUo2pDvXnSYjWTvJV7sZSvmqaWZMH2LfmpO-BTXHaEocujiGdGBR1xBjFwlRGq_o_2tRI62_iMCyoAqjUhUG24EbnScJVCxc7sed815n9ZhkyahHEBcuVE2pNR369xSZwuCaEzSngHm6S96Vf9BhO2Ysxk3FwP6dlKU3u2VrjxVTA; lang=vi; _admin_session_id=22972065-e46a-8a30-5a41-4b09bafa623c; .AspNetCore.Session=CfDJ8DA1016STPdNrAaTdK2HgWd3nFuS%2FNFRzK51VF5OxCtmvDYTQa1vX0B12XUuf5jno2mAdEfidZkyYn%2F3UsnzvQfwp6e8Ecc8uS%2B9PvAC%2BOnvfrpIYelimOiCwKXRbMgJafFXmRDZJAGMi4zTsBWRo15b7WrkYx5C3jXmVFXIziws; _ga_4NX0F91DEX=GS1.1.1727093848.48.1.1727093850.0.0.0; _ga=GA1.1.1357561029.1697356741; _ga_Z1V9QDBF2Q=GS1.1.1727093848.47.1.1727095593.0.0.0"
payload = {}
headers = {
  'Accept': 'application/json, text/plain, */*',
  'Accept-Language': 'vi',
  'Connection': 'keep-alive',
  'Cookie': cookie,
  'Referer': 'https://beanyfood.mysapogo.com/admin/orders?page=1&limit=20',
  'Sec-Fetch-Dest': 'empty',
  'Sec-Fetch-Mode': 'cors',
  'Sec-Fetch-Site': 'same-origin',
  'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/128.0.0.0 Safari/537.36',
  'X-Sapo-Client': 'sapo-frontend-v3',
  'X-Sapo-ServiceId': 'sapo-frontend-v3',
  'sec-ch-ua': '"Chromium";v="128", "Not;A=Brand";v="24", "Google Chrome";v="128"',
  'sec-ch-ua-mobile': '?0',
  'sec-ch-ua-platform': '"Windows"'
}

response = requests.request("GET", url, headers=headers, data=payload)


In [24]:
# load data
data: dict = json.loads(response.text)

In [25]:
created_ons = []
codes = []
customer_codes = []
customer_names = []
skus = []
product_names = []
lots_numbers = []
dates = []
units = []
quantities = []
gifts = []
prices_sapo = []
prices = []
prices_before_tax = []
stocks = []
discount_sales = []
discount_gifts = []
totals = []
unit_prices = []
paids = []
notes = []

In [26]:
for order in data["orders"]:
    if order["location_id"] != 598184:
        continue
    created_on = order["created_on"]
    created_on = datetime.strptime(created_on,"%Y-%m-%dT%H:%M:%SZ").strftime("%d/%m/%Y")
    code = order["code"]
    customer_code = order["customer_data"]["code"]
    customer_name = order["customer_data"]["name"]
    if "Finelife" in customer_name: continue
    paid = True
    if order["payment_status"] == "unpaid":
        paid = False
    stock = "CN HCM"
    for order_item in order["order_line_items"]:
        sku =  order_item["sku"]
        product_name = order_item["product_name"]
        unit = order_item["unit"]
        gift = 0
        price_sapo = order_item["price"]
        price_before_tax = price_sapo / 1.08
        price_before_tax = round(price_before_tax, 2)
        price = 0
        discount_sale = order_item["discount_rate"]    
        for lot in order_item["lots_dates"]:
            lots_number = lot["lots_number"]
            manufacture_date = datetime.strptime(lot["manufacture_date"],"%Y-%m-%dT%H:%M:%SZ").strftime("%d/%m")
            expiration_date = datetime.strptime(lot["expiration_date"],"%Y-%m-%dT%H:%M:%SZ").strftime("%d/%m")
            lots_date = manufacture_date + "-" + expiration_date
            quantity = int(lot["quantity"])
            total = quantity * price_sapo * (1-discount_sale/100)
            discount_gift = 0
            if total == 0:
                discount_gift = 100
                discount_sale = 0
            unit_price = total/quantity
            # save data 
            created_ons.append(created_on)
            codes.append(code)
            customer_codes.append(customer_code)
            customer_names.append(customer_name)
            skus.append(sku)
            product_names.append(product_name)
            lots_numbers.append(lots_number)
            dates.append(lots_date)
            units.append(unit)
            quantities.append(quantity)
            gifts.append(gift)
            prices_sapo.append(price_sapo)
            prices.append(price)
            prices_before_tax.append(price_before_tax)
            stocks.append(stock)
            discount_sales.append(discount_sale)
            discount_gifts.append(discount_gift)
            totals.append(total)
            unit_prices.append(unit_price)
            paids.append(paid)
            notes.append("")

In [27]:

data = {
    "created_on": created_ons,
    "code": codes,
    "customer_code": customer_codes,
    "customer_name": customer_names,
    "sku": skus,
    "product_name": product_names,
    "lots_number": lots_numbers,
    "date": dates,
    "unit": units,
    "quantity": quantities,
    "gift": gifts,
    "price_sapo": prices_sapo,
    "price": prices,
    "price_before_tax": prices_before_tax,
    "stock": stocks,
    "discount_sale": discount_sales,
    "discount_gift": discount_gifts,
    "total": totals,
    "unit_price": unit_prices,
    "paid": paids,
    "note": notes
}

# Create a DataFrame
df = pd.DataFrame(data)

In [28]:
df.to_csv('output.csv', index=False)